# GnssLogger Android App

This tutorial walks through how to parse the files obtained from [Google's GNSSLogger Android App](https://play.google.com/store/apps/details?id=com.google.android.apps.location.gps.gnsslogger&pcampaignid=web_share). Details for each data type can be found in Google's [gps-measurement-tools](https://github.com/google/gps-measurement-tools/blob/master/LOGGING_FORMAT.md) repository.

The toggle switches on the "Home" tab of the GNSSLogger app need to be set based on the type(s) of measurements you want to log: fixes, raw, NMEA, Rinex, sensors, etc.

After data is logged, the measurements can be saved immediately or retrieved at a later time within Android's internal storage. Data can be found on the phone under a directory similar to: ``<phone>/Internal shared storage/Android/data/com.google.android.apps.location.gps.gnsslogger/files/gnss_log/``

![GnssLogger screenshot](https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/docs/source/img/gnss_logger_app.jpg)

Load `gnss_lib_py` into the Python workspace

In [ ]:
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt

## Fix Measurements from gnss_log*.txt

The first type of measurements we can get is toggling a combination of "GNSS Location", "Fused Location", and/or "Network Location" in the GNSSLogger app. These location fixes are indicated with rows that start with "Fix" in the gnss_log.txt file output.

We start by downloading an example measurement file from the GNSSLogger Android App.

In [ ]:
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android/measurements/pixel6.txt --quiet -nc -O "../data/gnss_log.txt"

Load fix data into the existing `glp.AndroidRawFixes` class.

In [ ]:
fix_data = glp.AndroidRawFixes("../data/gnss_log.txt")

Based on the toggles you choose in the GNSSLogger app, you may have three different type of fixes: GNSS, network, and fused.

In [ ]:
fix_types = np.unique(fix_data["fix_provider"]).tolist()
fix_types

We will extract these three different type of fixes and rename their row names so that we can plot them as different colors all on the same map.

In [ ]:
fixes = []
for provider in fix_types:
    fix_provider = fix_data.where("fix_provider",provider)
    fix_provider.rename({"lat_rx_deg":"lat_rx_" + provider + "_deg",
                         "lon_rx_deg":"lon_rx_" + provider + "_deg",
                         "alt_rx_m":"alt_rx_" + provider + "_m",
                         }, inplace=True)
    fixes.append(fix_provider)

The GNSS, fused, and network location fixes are all shown in different colors in the map below.

In [ ]:
fig_fix = glp.plot_map(*fixes)
fig_fix.show()

Download example measurement file from the GNSSLogger Android App.

## Raw Measurements from gnss_log*.txt

The secibd type of measurements we can get is by toggling "Measurements" in the GNSSLogger app. These are raw measurements indicated with rows that start with "Raw" in the gnss_log.txt file output.

We start by loading our previously downloaded raw data into the existing `glp.AndroidRawGNSS` class. When you load the data here, behind the scenes it will compute the raw pseudorange values optionally filter measurements based on a variety of conditions. See the the [filter_raw_measurements](https://gnss-lib-py.readthedocs.io/en/latest/reference/parsers/android.html#android.AndroidRawGnss.filter_raw_measurements) reference documentation for details on the options for exisitng filter types.

In [ ]:
raw_data = glp.AndroidRawGnss(input_path="../data/gnss_log.txt", 
                              filter_measurements=True, 
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

We now have many data fields with which to work. Details for each data field can be found in `gnss_lib_py`'s [standard naming conventions](https://gnss-lib-py.readthedocs.io/en/latest/reference/reference.html#standard-naming-conventions) or Google's [gps-measurement-tools](https://github.com/google/gps-measurement-tools/blob/master/LOGGING_FORMAT.md#field-definitions) repository.

In [ ]:
raw_data.rows

Let's visualize the raw pseudoranges that have been compouted for us.

In [ ]:
figs = glp.plot_metric_by_constellation(raw_data,"gps_millis","raw_pr_m")

Before we can compute a Weighted Least Squares position, we first need to add satellite positions to our data. This block may take a bit of time since it has to download ephemeris files from the internet. Turn on `verbose=True` to see the data sources it's downloading.

In [ ]:
full_states = glp.add_sv_states(raw_data, source="precise", verbose=False)

We can then compute a "corrected" pseudorange by subtracting out each satellite's respective clock bias.

In [ ]:
full_states["corr_pr_m"] = full_states["raw_pr_m"] \
                         + full_states['b_sv_m']

Since we haven't yet computed any inter-constellation bias, we will crop down to only using GPS and Galileo measurements since the inter-constellation bias between them and GLONASS is quite large in this instance.

In [ ]:
full_states = full_states.where("gnss_id",("gps","galileo"))

We can now calculate our Weighted Least Squares position estimate.

In [ ]:
wls_estimate = glp.solve_wls(full_states)

Finally we'll plot our state estimate on a map.

In [ ]:
raw_fig = glp.plot_map(wls_estimate)
raw_fig.show()

## NMEA from gnss_log*.nmea

The third type of data that we can get from the GNSSLogger App is if "Log NMEA" is toggled. The NMEA data gives us a latitude and longitude directly.

We start by downloading an example NMEA log.

In [ ]:
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android/nmea/pixel6.nmea --quiet -nc -O "../data/gnss_log.nmea"

Load the NMEA data into the existing `glp.Nmea` class.

In [ ]:
nmea_data = glp.Nmea("../data/gnss_log.nmea")

We can plot the NMEA data on a map.

In [ ]:
nmea_fig = glp.plot_map(nmea_data)
nmea_fig.show()

We also have a few other fields explore.

In [ ]:
nmea_data.rows

For example, we can plot the number of satellites over time.

In [ ]:
fig = glp.plot_metric(nmea_data,"gps_millis","num_sats")

## Rinex Observations

The last type of GNSS log that we can obtain from the GNSSLogger app is Rinex observations. We can obtain this data from certain phones if the "Log RINEX" option is toggled in the GNSSLogger app.

We start by downloading an example rinex observation log.

In [ ]:
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android/rinex_obs/pixel6.23o --quiet -nc -O "../data/gnss_log.23o"

Load the NMEA data into the existing `glp.RinexObs` class.

In [ ]:
rinex_data = glp.RinexObs("../data/gnss_log.23o")

One small pecularity about rinex observations logged with the GNSSLogger app is that their timestamps don't account for the leapseconds since the start of the GPS epoch. Don't worry though, with `gnss_lib_py` that's a quick fix we can make.

In [ ]:
rinex_data["gps_millis"] -= glp.get_leap_seconds(rinex_data["gps_millis",0]) * 1E3

Before we can compute a Weighted Least Squares position, we first need to add satellite positions to our data. This block may take a bit of time since it has to download ephemeris files from the internet. Turn on `verbose=True` to see the data sources it's downloading.

In [ ]:
full_states = glp.add_sv_states(rinex_data, source="precise", verbose=False)

We can then compute a "corrected" pseudorange by subtracting out each satellite's respective clock bias.

In [ ]:
full_states["corr_pr_m"] = full_states["raw_pr_m"] \
                         + full_states['b_sv_m']

Since we haven't yet computed any inter-constellation bias, we will crop down to only using GPS and Galileo measurements since the inter-constellation bias between them and GLONASS is quite large in this instance.

In [ ]:
full_states = full_states.where("gnss_id",("gps","galileo"))

We can now calculate our Weighted Least Squares position estimate.

In [ ]:
wls_estimate = glp.solve_wls(full_states)

Finally we'll plot our state estimate on a map.

In [ ]:
raw_fig = glp.plot_map(wls_estimate)
raw_fig.show()

## Sensor Measurements gnss_log*.txt

If the "sensors" option is toggled in the GNSSLogger app, then the gnss_log*.txt log file will contain a number of sensor measurements that can be extracted with built in Python classes.

### Accelerometer Measurements from gnss_log*.txt

In [ ]:
raw_data = glp.AndroidRawAccel(input_path="gnss_log.txt")

In [ ]:
fig = glp.plot_metric(raw_data, "gps_millis","acc_x_uncal_mps2",label="x")
fig = glp.plot_metric(raw_data, "gps_millis","acc_y_uncal_mps2",label="y",fig=fig)
fig = glp.plot_metric(raw_data, "gps_millis","acc_z_uncal_mps2",label="z",fig=fig,
                     title="Accelerometer vs. GPS Time [milliseconds]")
ylabel = plt.ylabel("ACC UNCAL [m/s^2]")

### Gyro Measurements from gnss_log*.txt

In [ ]:
raw_data = glp.AndroidRawGyro(input_path="gnss_log.txt")

In [ ]:
fig = glp.plot_metric(raw_data, "gps_millis","ang_vel_x_uncal_radps",label="x")
fig = glp.plot_metric(raw_data, "gps_millis","ang_vel_y_uncal_radps",label="y",fig=fig)
fig = glp.plot_metric(raw_data, "gps_millis","ang_vel_z_uncal_radps",label="z",fig=fig,
                      title="Gyro vs. GPS Time [milliseconds]")
ylabel = plt.ylabel("ANG VEL UNCAL [rad/s]")

### Magnetometer Measurements from gnss_log*.txt

In [ ]:
raw_data = glp.AndroidRawMag(input_path="gnss_log.txt")

In [ ]:
fig = glp.plot_metric(raw_data, "gps_millis","mag_x_uncal_microt",label="x")
fig = glp.plot_metric(raw_data, "gps_millis","mag_y_uncal_microt",label="y",fig=fig)
fig = glp.plot_metric(raw_data, "gps_millis","mag_z_uncal_microt",label="z",fig=fig,
                      title="Magnetometer vs. GPS Time [milliseconds]")
ylabel = plt.ylabel("MAG UNCAL [micro T]")

### Orientation Measurements from gnss_log*.txt

In [ ]:
raw_data = glp.AndroidRawOrientation(input_path="gnss_log.txt")

In [ ]:
fig = glp.plot_metric(raw_data, "gps_millis","roll_rx_deg",label="roll")
fig = glp.plot_metric(raw_data, "gps_millis","pitch_rx_deg",label="pitch",fig=fig)
fig = glp.plot_metric(raw_data, "gps_millis","yaw_rx_deg",label="yaw",fig=fig,
                      title="Orientation [deg] vs. GPS Time [milliseconds]")
ylabel = plt.ylabel("Orientation [deg]")